In [57]:
import numpy as np
b=np.load('ny.eater-content-new2.npy', allow_pickle=True)
content=b.tolist()

In [2]:
# content

In [58]:
import pandas as pd 
pd.DataFrame(b).to_csv("ny.eater-content-new2.csv")

In [59]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [60]:
nlp = spacy.load('en_core_web_lg', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

C:\Users\godsh\anaconda3\envs\nlp_course\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\Users\godsh\anaconda3\envs\nlp_course\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)


In [61]:
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

new_content = []
for article in content:
    if article != []:
        text = ""
        for sent in article:
            text = text + sent
        new_content.append(text)

    

In [7]:
nlp.max_length = 3000200

In [8]:
# a=''
# for each in df_clean['clean']:
#     a += each


In [9]:
# doc1 = nlp(a)

In [10]:
# len(set(doc1))

In [11]:
# len(set(doc1))/len(a)

In [62]:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', re.sub("'s", '', re.sub("’s", '',str(article)))).lower() for article in new_content)

t = time()

txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_threads=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

Time to clean up everything: 0.35 mins


(412, 1)

In [72]:
brief_cleaning

<generator object <genexpr> at 0x0000028D29576848>

In [13]:
# df_clean[30:]

In [74]:
df_clean['clean'][0]

'jenny kwak be a chef at the top of -PRON- game move deftly between traditional korean and actually exciting fusion fare in park slopeamid new york ever globalize culinary scene a culture of cross pollination continue to thrive the city fine barbecue shack sell not just texas brisket but stellar jerk pork an ambitious indian spot in gramercy send out pitch perfect tandoori chicken and guacamole to boot an asian rotisserie in midtown stuff -PRON- breakfast pita with of all thing cream cheese and lox and now at haenyeo a knockout of a korean hangout in park slope the kitchen whip up bouillabaisse as masterful as that of any gallic temple chef owner jenny kwak infuse gochuchang chile powder into a classic french soup of fish fumet fennel tomato and pernod the rust color mixture zing -PRON- sinus with a pungent tang of the sea a filet of cod act as a steady conduit for the powerful broth as do a scattering of not something -PRON- d find in marseille glutinous rice cake mussel add a touch o

In [15]:
# wordNum=0
# count=0
# for j in df_clean['clean']:
#     wordNum =wordNum + len(j)
#     count = count + 1
# print(wordNum/count)    
    

  Using cached https://files.pythonhosted.org/packages/ef/92/325846f87c2ef1d6e5a6d54a55340877312b3afc4f23775448e3f97f3901/gensim-4.1.2-cp37-cp37m-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/89/61/0b316f1d669345003a7077ba9f7ebedd2b04eb164e3f42ba3c0b95469fc5/Cython-0.29.23-cp37-cp37m-win_amd64.whl
Note: you may need to restart the kernel to use updated packages.


In [19]:
from gensim.models.phrases import Phrases, Phraser

In [20]:
sent = [row.split() for row in df_clean['clean']]

In [21]:
bigram = Phrases(sent, min_count=30, progress_per=10000)

INFO - 07:32:14: collecting all words and their counts
INFO - 07:32:14: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 07:32:14: collected 256891 token types (unigram + bigrams) from a corpus of 545420 words and 412 sentences
INFO - 07:32:14: merged Phrases<256891 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 07:32:14: Phrases lifecycle event {'msg': 'built Phrases<256891 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 0.50s', 'datetime': '2021-11-24T07:32:14.745367', 'gensim': '4.1.2', 'python': '3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [22]:
sentences = bigram[sent]

In [23]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

23530

In [ ]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

In [24]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_lg

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

In [25]:
doc_list=sentences
# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

INFO - 07:32:27: adding document #0 to Dictionary(0 unique tokens: [])
INFO - 07:32:28: built Dictionary(23530 unique tokens: ['-PRON-', '-PRON-_re', 'a', 'about', 'accompaniment']...) from 412 documents (total 530362 corpus positions)
INFO - 07:32:28: Dictionary lifecycle event {'msg': "built Dictionary(23530 unique tokens: ['-PRON-', '-PRON-_re', 'a', 'about', 'accompaniment']...) from 412 documents (total 530362 corpus positions)", 'datetime': '2021-11-24T07:32:28.877065', 'gensim': '4.1.2', 'python': '3.7.7 (default, May  6 2020, 11:45:54) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


In [26]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=20, 
                                           random_state=2,
                                           update_every=1,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

INFO - 07:32:33: using autotuned alpha, starting with [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05]
INFO - 07:32:33: using symmetric eta at 0.05
INFO - 07:32:33: using serial LDA version on this node
INFO - 07:32:33: running online (multi-pass) LDA training, 20 topics, 10 passes over the supplied corpus of 412 documents, updating model once every 412 documents, evaluating perplexity every 412 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 07:32:35: -12.121 per-word bound, 4455.1 perplexity estimate based on a held-out corpus of 412 documents with 530362 words
INFO - 07:32:35: PROGRESS: pass 0, at document #412/412
INFO - 07:32:36: optimized alpha [0.09106496, 0.09232521, 0.092399396, 0.09631276, 0.054137222, 0.09529297, 0.09241304, 0.08772121, 0.09679173, 0.09404098, 0.09421782, 0.09471148, 0.09144975, 0.09704459, 0.09629294, 0.09759796, 0.09804256, 0.09594, 0.09663445, 0.08479934]
INFO

INFO - 07:32:53: topic #13 (0.078): 0.048*"the" + 0.040*"a" + 0.031*"of" + 0.028*"-PRON-" + 0.028*"and" + 0.026*"be" + 0.017*"to" + 0.015*"with" + 0.013*"in" + 0.009*"for"
INFO - 07:32:53: topic #15 (0.131): 0.056*"the" + 0.036*"a" + 0.034*"and" + 0.029*"be" + 0.029*"of" + 0.026*"-PRON-" + 0.019*"in" + 0.017*"with" + 0.014*"to" + 0.008*"that"
INFO - 07:32:53: topic #16 (0.244): 0.057*"the" + 0.039*"-PRON-" + 0.037*"a" + 0.031*"and" + 0.029*"of" + 0.022*"be" + 0.016*"with" + 0.016*"in" + 0.016*"to" + 0.010*"that"
INFO - 07:32:53: topic diff=0.715167, rho=0.377964
INFO - 07:32:55: -7.350 per-word bound, 163.2 perplexity estimate based on a held-out corpus of 412 documents with 530362 words
INFO - 07:32:55: PROGRESS: pass 6, at document #412/412
INFO - 07:32:56: optimized alpha [0.02787701, 0.027945457, 0.027949413, 0.045415834, 0.024058368, 0.028856339, 0.027950123, 0.027680382, 0.043604966, 0.028071107, 0.028432645, 0.028091773, 0.027898239, 0.07311907, 0.039499056, 0.12572117, 0.254842

In [27]:
pprint(lda_model.print_topics(num_words=20))

INFO - 07:33:12: topic #0 (0.023): 0.003*"the" + 0.002*"and" + 0.001*"-PRON-" + 0.001*"a" + 0.001*"of" + 0.001*"with" + 0.001*"be" + 0.001*"to" + 0.001*"in" + 0.001*"that" + 0.000*"for" + 0.000*"as" + 0.000*"on" + 0.000*"from" + 0.000*"but" + 0.000*"an" + 0.000*"like" + 0.000*"at" + 0.000*"restaurant" + 0.000*"or"
INFO - 07:33:12: topic #1 (0.023): 0.003*"the" + 0.002*"a" + 0.002*"and" + 0.002*"of" + 0.002*"-PRON-" + 0.002*"be" + 0.001*"with" + 0.001*"to" + 0.001*"that" + 0.001*"in" + 0.001*"on" + 0.001*"for" + 0.000*"at" + 0.000*"as" + 0.000*"an" + 0.000*"but" + 0.000*"restaurant" + 0.000*"i" + 0.000*"one" + 0.000*"like"
INFO - 07:33:12: topic #2 (0.023): 0.003*"the" + 0.003*"a" + 0.003*"-PRON-" + 0.002*"of" + 0.002*"and" + 0.002*"be" + 0.001*"to" + 0.001*"with" + 0.001*"in" + 0.001*"that" + 0.001*"at" + 0.001*"as" + 0.000*"for" + 0.000*"on" + 0.000*"an" + 0.000*"from" + 0.000*"or" + 0.000*"but" + 0.000*"more" + 0.000*"restaurant"
INFO - 07:33:12: topic #3 (0.038): 0.042*"-PRON-" + 0.

[(0,
  '0.003*"the" + 0.002*"and" + 0.001*"-PRON-" + 0.001*"a" + 0.001*"of" + '
  '0.001*"with" + 0.001*"be" + 0.001*"to" + 0.001*"in" + 0.001*"that" + '
  '0.000*"for" + 0.000*"as" + 0.000*"on" + 0.000*"from" + 0.000*"but" + '
  '0.000*"an" + 0.000*"like" + 0.000*"at" + 0.000*"restaurant" + 0.000*"or"'),
 (1,
  '0.003*"the" + 0.002*"a" + 0.002*"and" + 0.002*"of" + 0.002*"-PRON-" + '
  '0.002*"be" + 0.001*"with" + 0.001*"to" + 0.001*"that" + 0.001*"in" + '
  '0.001*"on" + 0.001*"for" + 0.000*"at" + 0.000*"as" + 0.000*"an" + '
  '0.000*"but" + 0.000*"restaurant" + 0.000*"i" + 0.000*"one" + 0.000*"like"'),
 (2,
  '0.003*"the" + 0.003*"a" + 0.003*"-PRON-" + 0.002*"of" + 0.002*"and" + '
  '0.002*"be" + 0.001*"to" + 0.001*"with" + 0.001*"in" + 0.001*"that" + '
  '0.001*"at" + 0.001*"as" + 0.000*"for" + 0.000*"on" + 0.000*"an" + '
  '0.000*"from" + 0.000*"or" + 0.000*"but" + 0.000*"more" + '
  '0.000*"restaurant"'),
 (3,
  '0.042*"-PRON-" + 0.038*"a" + 0.035*"the" + 0.031*"and" + 0.025*"be" 

In [28]:
lda_df = lda_model.get_document_topics(corpus,minimum_probability=0)
lda_df = pd.DataFrame(list(lda_df))
num_topics = lda_model.num_topics
lda_df.columns = ['Topic'+str(i) for i in range(num_topics)]
for i in range(len(lda_df.columns)):
    lda_df.iloc[:,i]=lda_df.iloc[:,i].apply(lambda x: x[1])
lda_df['Automated_topic_id'] =lda_df.apply(lambda x: np.argmax(x),axis=1)
lda_df.head()

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,...,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Automated_topic_id
0,0.000017,0.000017,0.000017,0.000028,0.000015,0.000017,0.000017,0.000017,0.000026,0.000017,...,0.000017,0.000017,0.366974,0.000025,0.000352,0.632347,0.000021,0.000026,0.000017,16
1,0.000019,0.000019,0.000019,0.000032,0.000017,0.000020,0.000019,0.000019,0.000029,0.000019,...,0.000019,0.000019,0.134963,0.000028,0.015889,0.848774,0.000024,0.000030,0.000019,16
2,0.000017,0.000017,0.000017,0.000028,0.000015,0.000017,0.000017,0.000017,0.000025,0.000017,...,0.000017,0.000017,0.008554,0.000024,0.000086,0.991036,0.000021,0.000026,0.000017,16
3,0.000016,0.000016,0.000016,0.000027,0.000014,0.000017,0.000016,0.000016,0.000025,0.000016,...,0.000016,0.000016,0.176904,0.000024,0.000084,0.822696,0.000020,0.000025,0.000016,16
4,0.000011,0.000011,0.000011,0.000018,0.000009,0.000011,0.000011,0.000011,0.000016,0.000011,...,0.000011,0.000011,0.007695,0.000015,0.000054,0.889787,0.102272,0.000016,0.000010,16


In [29]:
# Compute Perplexity, a measure of how good the model is. lower the better.
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
# Compute Coherence Score for lda model
coherence_model_lda = CoherenceModel(model=lda_model, texts=doc_list, dictionary=words, coherence='c_v')
coherence_lda_c_v = coherence_model_lda.get_coherence()
print('\nCoherence Score using c_v: ', coherence_lda_c_v)
# Compute Coherence Score for lda model
coherence_model_lda = CoherenceModel(model=lda_model, texts=doc_list, dictionary=words, coherence='u_mass')
coherence_lda_u_mass = coherence_model_lda.get_coherence()
print('\nCoherence Score using u_mass: ', coherence_lda_u_mass)

INFO - 07:33:23: -7.309 per-word bound, 158.6 perplexity estimate based on a held-out corpus of 412 documents with 530362 words
INFO - 07:33:23: using ParallelWordOccurrenceAccumulator(processes=7, batch_size=64) to estimate probabilities from sliding windows



Perplexity:  -7.309387651829852


INFO - 07:33:27: 1 batches submitted to accumulate stats from 64 documents (79180 virtual)
INFO - 07:33:27: 2 batches submitted to accumulate stats from 128 documents (161481 virtual)
INFO - 07:33:28: 3 batches submitted to accumulate stats from 192 documents (242227 virtual)
INFO - 07:33:28: 4 batches submitted to accumulate stats from 256 documents (319383 virtual)
INFO - 07:33:28: 5 batches submitted to accumulate stats from 320 documents (391983 virtual)
INFO - 07:33:28: 6 batches submitted to accumulate stats from 384 documents (455620 virtual)
INFO - 07:33:28: 7 batches submitted to accumulate stats from 448 documents (485454 virtual)
INFO - 07:33:30: 7 accumulators retrieved from output queue
INFO - 07:33:30: accumulated word occurrence stats for 485454 virtual documents



Coherence Score using c_v:  0.22856393850536377

Coherence Score using u_mass:  -0.19573664217533498


In [38]:
%pip install bokeh

  Using cached https://files.pythonhosted.org/packages/05/8e/8de486cbd03baba4deef4142bd643a3e7bbe954a784dc1bb17142572d127/packaging-21.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/17/61/32c3ab8951142e061587d957226b5683d1387fb22d95b4f69186d92616d1/typing_extensions-4.0.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


In [44]:
from pyLDAvis import gensim_models as gensim
from time import time 
import numpy as np
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast

import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer
from textblob import TextBlob
import scipy.stats as stats
from scipy.sparse import csr_matrix
import spacy

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
output_notebook()


%matplotlib inline

Loading BokehJS ...

In [45]:
import pyLDAvis

In [46]:
a=pyLDAvis.gensim_models.prepare(lda_model, corpus, words)

C:\Users\godsh\anaconda3\envs\nlp_course\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [53]:
pyLDAvis.display(a)

In [48]:
import multiprocessing

from gensim.models import Word2Vec

In [49]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [52]:
w2v_model = Word2Vec(min_count=20,
                     window=20,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=3)

TypeError: __init__() got an unexpected keyword argument 'size'

In [ ]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
w2v_model.init_sims(replace=True)

In [ ]:
w2v_model.wv.most_similar(positive=["beef","chicken","fish","short_rib","steak"])

In [ ]:
all=[]
for i in sent:
    all +=i


In [ ]:
for each in w2v_model.predict_output_word(['steak'],100):
    if nlp(each[0])[0].pos_ == 'VERB' or nlp(each[0])[0].pos_ == 'ADJ': #NOUN,VERB
        print (each)

In [ ]:
import numpy as np
labels = []
count = 0
max_count = 20
X = np.zeros(shape=(max_count,len(w2v_model['chicken'])))

for term in w2v_model.wv.vocab:
    X[count] = w2v_model[term]
    labels.append(term)
    count+= 1
    if count >= max_count: break

# It is recommended to use PCA first to reduce to ~50 dimensions
from sklearn.decomposition import PCA
pca = PCA(n_components=20)
X_50 = pca.fit_transform(X)

# Using TSNE to further reduce to 2 dimensions
from sklearn.manifold import TSNE
model_tsne = TSNE(n_components=2, random_state=0)
Y = model_tsne.fit_transform(X_50)

# Show the scatter plot
import matplotlib.pyplot as plt
plt.scatter(Y[:,0], Y[:,1], 20)

# Add labels
for label, x, y in zip(labels, Y[:, 0], Y[:, 1]):
    plt.annotate(label, xy = (x,y), xytext = (0, 0), textcoords = 'offset points', size = 10)

plt.show()

In [ ]:
w2v_model.wv.most_similar(positive=["beef","chicken","pork"], negative=["western"], topn=3)


In [ ]:
w2v_model.wv.most_similar(positive=["beef"], topn=20)

In [ ]:
for i in range(20):
     print(w2v_model.wv.most_similar(positive=["beef"], topn=20)[i][0], end = ', ')

In [ ]:
w2v_model.wv.most_similar(positive=["beef","noodle","sushi"],negative=["chicken"], topn=20)

In [ ]:
for each in w2v_model.wv.most_similar(positive=["beef","noodle","sushi"],negative=["chicken"], topn=20):
    print (each[0], end = ', ')

In [ ]:
count=0
for each in w2v_model.wv.most_similar(positive=["beef","noodle","sushi"],negative=["chicken"], topn=50):
    if nlp(each[0])[0].pos_ != 'NOUN': #NOUN,VERB
        if count < 20:
            print (each[0], end = ', ')
            count += 1

In [ ]:
def simlar_pos(words):
    count=0
    list=[]
    for each in w2v_model.wv.most_similar(positive=words, topn=100):
        if nlp(each[0])[0].pos_ == 'NOUN': #NOUN,VERB
            if count < 20:
                list.append(each)
                count += 1
    return list

In [ ]:
simlar_pos(["beef","noodle","sushi"])

In [ ]:
a=[5,6,7]
print(a*3)

In [ ]:
print(simlar_pos("bread"))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [ ]:
def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=15).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('visualization for {}'.format(word.title()))

In [ ]:
def tsnescatterplot1(model, word1,word2):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = []
    color_list  = []

    # adds the vector of the query word
    for wrd_score in word1:
        wrd_vector = model.wv.__getitem__([wrd_score])
        word_labels.append(wrd_score)
        color_list.append('red')
        arrays = np.append(arrays, wrd_vector, axis=0)

    
    # gets list of most similar words
    close_words = model.wv.most_similar(positive=word1,negative=word2, topn=20)
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    

        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=21).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('Similarity visualization for {} without {}'.format(word1.title(),word2.title()))

In [ ]:
def tsnescatterplot2(model, word1,title):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = []
    color_list  = []

    # adds the vector of the query word
    for wrd_score in word1:
        wrd_vector = model.wv.__getitem__([wrd_score])
        word_labels.append(wrd_score)
        color_list.append('red')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar(positive=word1,topn=20)
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    

        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=len(word1)+20).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title(title)

In [ ]:
def tsnescatterplot3(model, word1,title):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = []
    color_list  = []

    # adds the vector of the query word
    for wrd_score in word1:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score)
        color_list.append('red')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar(positive=word1,topn=20)
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    

        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=len(word1)+20).fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title(title)

In [ ]:
tsnescatterplot1(w2v_model, ["beef"],[])


In [ ]:
#tsnescatterplot(w2v_model, 'beef', [i[0] for i in w2v_model.wv.most_similar(negative=["beef"])])

In [ ]:
tsnescatterplot(w2v_model, 'chicken', [i[0] for i in w2v_model.wv.most_similar(positive=["fish"])])

In [ ]:
tsnescatterplot(w2v_model, 'dumpling', [i[0] for i in w2v_model.wv.most_similar(positive=["bread"])])

In [ ]:
tsnescatterplot(w2v_model, 'masala', [i[0] for i in w2v_model.wv.most_similar(positive=["soy_sauce"])])

In [ ]:
1. catagory
2. pos & neg
3. top ten similarity

In [ ]:
unik word,more details about data, how many artical mention chicken, beef
most frequent word.
after clearing, explain dataset, compair how many is left , how many "beef" remaining

explain figure 2
explain figure TSNE or PCA
explain figure